# Final assignment

## Load some libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.morphology as morph
import skimage.measure as measure
from skimage import filters
from scipy import ndimage
%matplotlib inline

## Assignment

You have an image with round regions that you want to identify and create an image with only these features visible. In this assignment is a review of the lectures and previous assignments understanding the steps and solving the tasks is a good preparation for the exam.

1. Histograms
2. Filters
3. Thresholding
4. Cleaning
5. Labeling
6. Measure region properties
7. Classification based on abstract properties
8. Item extraction


### Step 1: Load image
Load the image 'psi_slice_noisy.tif' and show it

In [ ]:
# your code here

### Step 2: Look at the histogram

Compute and plot the histogram here

In [2]:
 # your code here

The image looks rather noisy... i.e. it has low SNR. Improve the SNR using a filter. Show the image and its histogram after filtering. The histogram is also incomplete on the left sie, i.e. the noise distribution is truncated.

In [ ]:
fimg=              # apply the filter of your choice here
plt.subplot(1,2,1)
plt.imshow(fimg)

plt.subplot(1,2,2)
                 # compute and plot the histogram of fimg


Get a threshold using an automatic method provided by SciKit Image and apply to the image

In [ ]:
otsu =                             # compute the otsu threshold here
print('The threshold according to',otsu)
bimg =                             # apply the Otsu threshold on fimg here                                    
plt.imshow(bimg)
plt.title('Otsu')

As you see this threshold only separates background from sample. What we need is to compute the threshold from the sample pixels only. The function below helps you doing this.

In [ ]:
def maskedOtsu(img,mask) :
    data=[]
    dims=img.shape
    for r in np.arange(0,dims[1]) :
        for c in np.arange(0,dims[0]) :
            if (mask[r,c]!=0) :
                data.append(img[r,c])
            
    ndata=np.asarray(data)
    return filters.threshold_otsu(ndata)


Use the previous bi-level image as mask and find a useful threshold with maskedOtsu. 

In [ ]:
otsu2=maskedOtsu(fimg,bimg)
bimg2=                    # apply the masked threshold on fimg here

plt.imshow(bimg2)

In bimg2 some pixels are misclassified. Use a morphological operation to clean this.

In [ ]:
clean=               # apply your cleaning operation here
plt.imshow(clean)

### Step 3: Labeling

Now, the image is ready to label. 

__Q1__ For this you have two choices connected pixels labeling or watershed. Which makes more sense here?

In [ ]:
lbl=             # write your labeling code here
plt.imshow(lbl)

No matter which method you chose. More regions than the round ones are labeled. We have to find a method to classify which ones we want to keep.

### Step 4: Identify the discs

The first step is to measure some properties of different items. Use the library scikit.image function region properties to make measurements on the segmented regions.

http://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops  

Select pairs of scalar properties from the list and decide if they help you to identify the disks.

In [ ]:
m=measure.regionprops(lbl)
p1=[]
p2=[]
for prop in m:
    p1.append(prop.property1) # enter a property here
    p2.append(prop.property2) # enter a different property here
    
plt.plot(p1,p2,'+')
plt.xlabel('Property 1')
plt.ylabel('Property 2')

Selecting the item is done by applying thresholds on each property. Is done with help of the function below. Plese complete the test in the function  

In [ ]:
def selectItems(m,p1,p2,th1,th2) :
    selection = []
    for i in np.arange(0,len(p1)) :
        if ((th1<p1[i]) & (th2<p2[i])) : # here you may have to modify the tests
            selection.append(m[i])
            
    return selection

In [ ]:
# Enter you thresholds here to test if the function works
th1= # enter threshold for property 1 here
th2= # enter threshold for property 2 here
selection=selectItems(m,p1,p2,th1,th2) 
print('Found 14 items',len(selection)==14)

### Step 5: Isolate the disks

Morphological reconstruction is a method to extract the regions. For this to work you need to create a marker image using the positons of the disks. The marker is zero valued except on the centroids of each disk which has the value of disk label.

__Q2__ What is the function of the reconstruction operator here?

In [ ]:
marker=np.zeros(lbl.shape) # We start with a zero valued image

for prop in selection :
    x=np.round(prop.centroid[0]).astype(int)
    y=np.round(prop.centroid[1]).astype(int)
    marker[x,y]=        # enter a marker value here
    
final=morph.reconstruction(marker,lbl)
plt.imshow(final)